In [4]:
# imports
import jax
from flax import nnx
import optax
import jax.numpy as jnp
import dataclasses

from typing import List

In [5]:
jax.devices()

[CudaDevice(id=0)]

# Configuration
This model has more parameters, let's create a special class with configuration parameters

# Configuration
This model has more parameters, let's create a special class with configuration parameters

In [10]:
@dataclasses.dataclass
class Config:
    embeded_size:int = 64

    # Image-related parameters
    in_channels:int = 3
    image_width:int = 256
    image_height:int = 256

    # Architecture related parameters
    num_residual_hiddens:int = 128
    num_residual_input:int = 3
    # Dataset-related parameters
    dataset_name:str = 'bitmind/ffhq-256'
    

In [25]:
class ResBlock(nnx.Module):
    """One block of the encoder with a residual connection"""
    def __init__(self, config:Config, rngs: nnx.Rngs):
        super().__init__()
        self.config= config
        self.conv1 = nnx.Conv(in_features=config.num_residual_input, out_features=config.num_residual_hiddens, kernel_size=(3,3), strides=1, padding='SAME', rngs=rngs)
        self.conv2 = nnx.Conv(in_features=config.num_residual_hiddens, out_features=config.num_residual_input, kernel_size=(1,1), strides=1, padding='SAME', rngs=rngs)
    def __call__(self, inputs:jax.Array):
        out = self.conv1(inputs)
        out = jax.nn.relu(out)
        out = self.conv2(out)
        out = jax.nn.relu(out)
        return out+inputs
    

In [26]:
rngs = nnx.Rngs(0)
config = Config()
res_block = ResBlock(config, rngs)

In [29]:
test_image=jnp.ones((config.image_height, config.image_width, 3))
test_image.shape, res_block(test_image).shape            

((256, 256, 3), (256, 256, 3))

In [ ]:
class Encoder(nnx.Module):
    """VQVAE encoder"""
    def __init__(self,config):
        super().__init__()
        self.config = config
        
    